## NBA Players Data Webscraping to Pandas Dataframe to SQLite Database

### Necessary Imports

In [1]:
import sqlite3
import pandas as pd

### Team Names + Abbreviations Dictionaries

In [2]:
team_names =     ['Utah Jazz', 'Sacramento Kings', 'Washington Wizards',
                  'Boston Celtics', 'Milwaukee Bucks', 'Oklahoma City Thunder',
                  'Chicago Bulls', 'Phoenix Suns', 'Philadelphia 76ers',
                  'New Orleans Pelicans', 'Charlotte Hornets', 'Los Angeles Lakers',
                  'Indiana Pacers', 'Toronto Raptors', 'Cleveland Cavaliers',
                  'Denver Nuggets', 'Minnesota Timberwolves', 'Brooklyn Nets',
                  'San Antonio Spurs', 'Dallas Mavericks', 'Houston Rockets',
                  'Detroit Pistons', 'Portland Trail Blazers', 'Atlanta Hawks',
                  'Golden State Warriors', 'Miami Heat', 'Los Angeles Clippers',
                  'New York Knicks', 'Memphis Grizzlies', 'Orlando Magic']

team_abbrs =     ['UTAH', 'SAC', 'WAS', 'BOS', 'MIL', 'OKC', 'CHI', 'PHX', 'PHI',
                  'NO', 'CHA', 'LAL', 'IND', 'TOR', 'CLE', 'DEN', 'MIN', 'BKN',
                  'SAS', 'DAL', 'HOU', 'DET', 'POR', 'ATL', 'GSW', 'MIA', 'LAC',
                  'NYK', 'MEM', 'ORL']

team_name_abbr_dict = {}
team_abbr_name_dict = {}

for i in range(len(team_names)):
    team_name_abbr_dict[team_names[i]]=team_abbrs[i]

for i in range(len(team_abbrs)):
    team_abbr_name_dict[team_abbrs[i]]=team_names[i]

### Webscrape All NBA Players Data into Pandas Dataframe

In [3]:
all_players = pd.DataFrame()

for i in team_abbrs:
    team_name = team_abbr_name_dict[i].lower().replace(" ","-")
    players = pd.read_html(f'https://www.espn.com/nba/team/roster/_/name/{i.lower()}/{team_name}')[0]
    players = players.drop(columns='Unnamed: 0')
    players[['Name', 'Number']] = players['Name'].str.extract('(\D+)(\d+)', expand=True)

    col_name="Number"
    num_col = players.pop(col_name)
    players.insert(1, col_name, num_col)
    
    players.insert(0, 'TEAM', i)
    
    all_players = all_players.append(players)

all_players = all_players.reset_index(drop=True)

### Convert to All Numeric Values

#### Height to Inches

In [4]:
def height_to_inches(height_str):
    feet, inches = height_str.split("' ")
    return int(feet) * 12 + int(inches[:-1])

In [5]:
all_players.HT = all_players.HT.apply(height_to_inches)
all_players = all_players.rename(columns = {'HT':'HT_in'})

#### Player Weights

In [6]:
all_players.WT = all_players.WT.str.rstrip(" lbs")
all_players = all_players.rename(columns = {'WT':'WT_lb'})

#### Player Salary

In [7]:
try:
    all_players.Salary = all_players.Salary.str.replace('[\$\,]|\.\d*', '',regex=True).astype(int)
except ValueError:
    pass

### Save DataFrame to CSV for Future Usage

In [8]:
all_players.to_csv('NBA_Player_Bios.csv')

### Optional: Display Player Dataframe to Ensure Webscraping and Conversion was Succesful

In [9]:
#pd.set_option('display.max_rows', 500)
#all_players

### Define and Connect Database and Cursor

In [10]:
db = sqlite3.connect('Databases/NBA_Players.db')
cur = db.cursor()

### Define NBA Players SQL Database Table and Column Values

In [11]:
cur.execute('''
    CREATE TABLE IF NOT EXISTS NBA_Players(
        team text NOT NULL,
        name text PRIMARY KEY,
        num integer,
        pos text NOT NULL,
        age integer NOT NULL,
        ht_in integer NOT NULL,
        wt_lb integer NOT NULL,
        college text,
        salary integer);
''')

### Convert Player DataFrame into a List for SQL Table Insertion

In [12]:
players_list = list(all_players.itertuples(index=False, name=None))
#players_list

### Insert Player List Data into SQL Database

In [13]:
cur.executemany('''
        INSERT INTO NBA_Players(team,name,num,pos,age,ht_in,wt_lb,college,salary)
        VALUES(?,?,?,?,?,?,?,?,?)''', players_list)

### Commmit DataBase ~ ends a transaction in RDBMS and makes all changes visible to other users

In [14]:
db.commit()

### Close Any Open Cursors

In [15]:
db.close()